# Save all links

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import requests
import re
import time

# open browser
browser = webdriver.Chrome()
browser.get('http://www.yes123.com.tw/admin/index.asp')

# input search keywords
browser.find_element_by_id('find_key1').clear()
browser.find_element_by_id('find_key1').send_keys('軟體 工程師')

time.sleep(2)
# Click search button
browser.find_element_by_class_name('n_serch_btn').click()

time.sleep(2)
counter = 0

while True:
    try:
        counter += 1
        browser.find_element_by_tag_name('body').send_keys(Keys.END)
        links = [link.get_attribute('href') for link in browser.find_elements_by_class_name('jobname')]

        with open('link_yes123_2.csv', 'w') as f:
            f.write('\n'.join(links) + '\n')
        print('[Debug] links of page %d are saved' % (counter))
        browser.find_element_by_link_text('>').click()
        time.sleep(1)   
    except Exception as e:
        print(e)
        break
print('The end of pages is No.%d' %(counter))
print('Browser will quit!')
browser.quit()

# Get the content form one 
page


In [ ]:
def save_content(link):
    import requests 
    from bs4 import BeautifulSoup
    import re
    res = requests.get(link)
    soup = BeautifulSoup(res.text, 'lxml')
    content = ''
#     print(res)
    
    divs = soup.select('div.comp_detail')
#     print(len(divs))
    del divs[-1]
    for div in divs:
        content += div.text.replace('\n', '').replace('\xa0', '')

    text = '/'.join(re.findall('object c|visual basic|\.net|[A-Za-z]+[+#]*', content, re.IGNORECASE))

    with open('yes123_job3.csv', 'a') as f:
        f.write(text + '\n')


# Save all contents


In [ ]:
import time

with open('link_yes123.csv', 'r') as fr:
    link_list = fr.read().split('\n')
for idx, link in enumerate(link_list):
    if idx % 50 == 0:
        print('Current link {}'.format(idx))
    try:
        save_content(link)
#         time.sleep(0.3)
    except Exception as e:
        print(e)
        continue

# Multithreading

In [ ]:

import time

import threading
with open('link_yes123.csv', 'r') as fr:
    link_list = fr.read().split('\n')
with open('yes123_job2.csv', 'a') as f:
    for idx, link in enumerate(link_list):
        if idx % 50 == 0:
            print('Current link {}'.format(idx))
        try:
            lock = threading.Lock()
            th = threading.Thread(target=save_content_th, args=(link, lock, f))
            th.start()
            time.sleep(0.1)
        except Exception as e:
            print(e)
            continue
    else:
        print('Process finished')

In [ ]:
def save_content_th(link, lock, f):
    import requests 
    from bs4 import BeautifulSoup
    import re
    import time
    res = requests.get(link)
#     time.sleep(1)
    soup = BeautifulSoup(res.text, 'lxml')
    content = ''
    
    divs = soup.select('div.comp_detail')
#     del divs[-1]
    
    for div in divs:
        content += div.text.replace('\n', '').replace('\xa0', '')
    
    text = '/'.join(re.findall('object c|visual basic|\.net|[A-Za-z]+[+#]*', content, re.IGNORECASE))
    lock.acquire()
    f.write(text + '\n')
    lock.release()

# Word Counts

In [ ]:
# counter
from collections import Counter

counter = Counter()
with open('yes123_job.csv', 'r') as fr:
    for line in fr:
        if len(line) > 0:
            words= list(set(line.strip().split('/')))  # remove duplicate items in line
        for word in words:
            if word.lower() in counter.keys():
                counter[word.lower()] += 1
            else:
                counter[word.lower()] =  1


In [ ]:
counter.most_common(50)
with open('123.csv', 'w') as f:
    for lan, counts in counter.most_common():
        f.write('{},{}\n'.format(lan,counts))